#### Instructions:  
1. Libraries allowed: **Python basic libraries, numpy, pandas, scikit-learn (only for data processing), pytorch, and ClearML.**
2. Show all outputs.
3. Submit jupyter notebook and a pdf export of the notebook. Check canvas for detail instructions for the report. 
4. Below are the questions/steps that you need to answer. Add as many cells as needed. 

## Step 4: hyperparameter tuning without learning rate decay
Do hyperparater tuning with ClearML and copy the plots (e.g., parallel coordinates) from ClearML and visualize them here.

In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from clearml import Task
import torch.nn.functional as F

# Initialize ClearML Task
task = Task.init(project_name="Hyperparameter Tuning", task_name="With Learning Rate Scheduler")

# Define hyperparameters
hyperparams = {
    "num_layers": 3,
    "filters": [32, 64, 128],
    "learning_rate": [0.05, 0.08, 0.11],  # Refined learning rates
    "batch_size": 128,
    "num_epochs": 5,  # Increased epochs
    "momentum": [0.7, 0.8],
    "weight_decay": 0.001
}
task.connect(hyperparams)



ClearML Task: overwriting (reusing) task id=1c991bf1927f46c992a9ae8addc6f892
2024-11-26 11:29:54,200 - clearml.Repository Detection - WARNING - Password protected Jupyter Notebook server was found! Add `sdk.development.jupyter_server_password=<jupyter_password>` to ~/clearml.conf
2024-11-26 11:29:54,229 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/cc76c0e26e4f47f99330259096e45abd/experiments/1c991bf1927f46c992a9ae8addc6f892/output/log


{'num_layers': 3,
 'filters': [32, 64, 128],
 'learning_rate': [0.05, 0.08, 0.11],
 'batch_size': 128,
 'num_epochs': 5,
 'momentum': [0.7, 0.8],
 'weight_decay': 0.001}

In [2]:
# Dataset preparation with augmentation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder('train', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_subset, val_subset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=hyperparams["batch_size"], shuffle=True)
val_loader = DataLoader(val_subset, batch_size=hyperparams["batch_size"], shuffle=False)



In [3]:
# Define model
class CNN(nn.Module):
    def __init__(self, num_classes=7):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# Training and validation function with learning rate scheduler
def train_and_validate(lr, momentum):
    model = CNN(num_classes=7).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    patience = 3  # Early stopping patience
    no_improvement = 0  # Track epochs without improvement
    best_model = None

    for epoch in range(hyperparams["num_epochs"]):
        model.train()
        running_loss = 0.0

        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_loss /= len(val_loader)
        accuracy = correct / total

        # Log metrics to ClearML
        task.get_logger().report_scalar("Loss", "Validation", val_loss, epoch)
        task.get_logger().report_scalar("Accuracy", "Validation", accuracy, epoch)

        print(f"Epoch {epoch+1}/{hyperparams['num_epochs']}, "
              f"Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}")

        # Scheduler step
        scheduler.step(val_loss)

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model
            no_improvement = 0  # Reset patience counter
        else:
            no_improvement += 1

        # Early stopping
        if no_improvement >= patience:
            print("Early stopping triggered.")
            break

    return best_model

In [5]:
# Hyperparameter tuning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for lr in hyperparams["learning_rate"]:
    for momentum in hyperparams["momentum"]:
        print(f"Training with LR={lr}, Momentum={momentum}")
        best_model = train_and_validate(lr, momentum)

# Save the best model to a file
torch.save(best_model.state_dict(), "best_model.pth")
print("Best model saved.")


Training with LR=0.05, Momentum=0.7
Epoch 1/5, Train Loss: 1.7343, Val Loss: 1.6478, Val Accuracy: 0.3382
Epoch 2/5, Train Loss: 1.6073, Val Loss: 1.5494, Val Accuracy: 0.3945
Epoch 3/5, Train Loss: 1.5161, Val Loss: 1.4922, Val Accuracy: 0.4232
Epoch 4/5, Train Loss: 1.4387, Val Loss: 1.4139, Val Accuracy: 0.4532
Epoch 5/5, Train Loss: 1.3774, Val Loss: 1.3928, Val Accuracy: 0.4699
Training with LR=0.05, Momentum=0.8
Epoch 1/5, Train Loss: 1.7248, Val Loss: 1.6468, Val Accuracy: 0.3304
Epoch 2/5, Train Loss: 1.5663, Val Loss: 1.5340, Val Accuracy: 0.4000
Epoch 3/5, Train Loss: 1.4481, Val Loss: 1.4368, Val Accuracy: 0.4443
Epoch 4/5, Train Loss: 1.3744, Val Loss: 1.3652, Val Accuracy: 0.4711
Epoch 5/5, Train Loss: 1.3100, Val Loss: 1.3441, Val Accuracy: 0.4796
Training with LR=0.08, Momentum=0.7
Epoch 1/5, Train Loss: 1.7128, Val Loss: 1.6555, Val Accuracy: 0.3607
Epoch 2/5, Train Loss: 1.5524, Val Loss: 1.5047, Val Accuracy: 0.4056
Epoch 3/5, Train Loss: 1.4462, Val Loss: 1.4718, Val

## Step 5: hyperparameter tuning with learning rate decay
Do hyperparater tuning with ClearML and copy the plots (e.g., parallel coordinates) from ClearML and visualize them here.

In [6]:
from torch.optim.lr_scheduler import StepLR
from clearml import Task
import torch

# Close any existing ClearML task
if Task.current_task():
    Task.current_task().close()

# Initialize new ClearML task
task = Task.init(project_name="Hyperparameter Tuning", task_name="With Learning Rate Decay")


Password protected Jupyter Notebook server was found! Add `sdk.development.jupyter_server_password=<jupyter_password>` to ~/clearml.conf


ClearML Task: created new task id=958c04ab27bb4669bf1c2c0268b74847
ClearML results page: https://app.clear.ml/projects/cc76c0e26e4f47f99330259096e45abd/experiments/958c04ab27bb4669bf1c2c0268b74847/output/log


In [7]:
# Update training loop to include scheduler and save the best model
def train_and_validate_with_lr_decay(lr, momentum, weight_decay, step_size, gamma):
    model = CNN(num_classes=7).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    patience = 3  # Stop if no improvement for 3 epochs
    no_improvement = 0
    best_model = None

    for epoch in range(hyperparams["num_epochs"]):
        model.train()
        running_loss = 0.0

        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Step the scheduler
        scheduler.step()

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_loss /= len(val_loader)
        accuracy = correct / total

        print(f"Epoch {epoch+1}/{hyperparams['num_epochs']}, "
              f"Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}")

        # Check for improvement
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model
            no_improvement = 0  # Reset patience counter
        else:
            no_improvement += 1

        # Early stopping
        if no_improvement >= patience:
            print("Early stopping triggered.")
            break

    # Save the best model after training
    if best_model is not None:
        torch.save(best_model.state_dict(), "best_model_with_lr_decay.pth")
        print("Best model saved as 'best_model_with_lr_decay.pth'.")

    return best_model

In [8]:
# Hyperparameter tuning loop
for lr in hyperparams["learning_rate"]:
    for momentum in hyperparams["momentum"]:
        best_model = train_and_validate_with_lr_decay(
            lr, momentum, hyperparams["weight_decay"], step_size=5, gamma=0.5
        )

Epoch 1/5, Train Loss: 1.7282, Val Loss: 1.7214, Val Accuracy: 0.2997
Epoch 2/5, Train Loss: 1.6042, Val Loss: 1.5485, Val Accuracy: 0.4094
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
Epoch 3/5, Train Loss: 1.5031, Val Loss: 1.4590, Val Accuracy: 0.4383
Epoch 4/5, Train Loss: 1.4352, Val Loss: 1.4285, Val Accuracy: 0.4500
Epoch 5/5, Train Loss: 1.3802, Val Loss: 1.4039, Val Accuracy: 0.4573
Best model saved as 'best_model_with_lr_decay.pth'.
Epoch 1/5, Train Loss: 1.7217, Val Loss: 1.6522, Val Accuracy: 0.3316
Epoch 2/5, Train Loss: 1.5593, Val Loss: 1.5488, Val Accuracy: 0.3976
Epoch 3/5, Train Loss: 1.4617, Val Loss: 1.4946, Val Accuracy: 0.4173
Epoch 4/5, Train Loss: 1.3973, Val Loss: 1.4150, Val Accuracy: 0.4664
Epoch 5/5, Train Loss: 1.3424, Val Loss: 1.3754, Val Accuracy: 0.4751
Best model saved as 'best_model_with_lr_decay.pth'.
Epoch 1/5, Train Loss: 1.7059, Val Loss: 1.6219, Val Accuracy: 0.3736
Epoch 2/5, Train Loss:

In [9]:
torch.save(best_model.state_dict(), "best_model_with_lr_decay.pth")


## Step 6: Evaluation
Evaluate the best model on test dataset and report accuracy, precision, recall, and F1 score.

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load test dataset
test_dataset = datasets.ImageFolder('test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=hyperparams["batch_size"], shuffle=False)

# Recreate the model and load the saved best model weights
best_model = CNN(num_classes=7).to(device)  # Instantiate the model
best_model.load_state_dict(torch.load("best_model_with_lr_decay.pth"))  # Load the saved weights
best_model.eval()  # Set the model to evaluation mode

# Evaluate the model
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)  # Use the best_model here
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.4926
Precision: 0.4871
Recall: 0.4150
F1 Score: 0.4056


## Step 7: Analysis
Provide a complete analysis of the whole process. 

Answer: 

## Part-1

### Objective:
The project aimed to develop a robust pipeline for emotion detection using CNNs, focusing on data preprocessing, model training, and hyperparameter tuning with ClearML.

#### Data Preprocessing:

Normalized dataset (Mean: 0.5077, Std: 0.2119).
Applied augmentations (rotation, flipping) to improve generalization.
Partitioned data into training, validation, and test sets.
#### Model Training:

Developed a DynamicCNN with configurable layers and regularization (dropout, weight decay).
Used SGD and Adam optimizers, with learning rate decay to enhance convergence.
Overfitted a small subset (two samples) to validate model capacity.
#### Hyperparameter Tuning:

Tested learning rates, momentum, and weight decay.
Optimal parameters identified: Learning rate: 0.05–0.08, Weight decay: 1e-4.
ClearML facilitated experiment tracking and visualization.
#### Performance:

DynamicCNN achieved Validation Accuracy: 33.56% after 10 epochs.
Overfitting on two samples reached 100% accuracy in one epoch.
### Challenges:
Overfitting in smaller datasets.
Validation accuracy plateaued, requiring more data or advanced architectures.


## Part-2
### Objective:
Develop a CNN for multi-class emotion detection with PyTorch and ClearML for hyperparameter tuning.

#### Data Preparation:

Dataset split into training, validation, and test sets.
Applied transformations: grayscale conversion, resizing, normalization, and augmentations (flipping, rotation).
#### Model Development:

A configurable CNN with 2 convolutional layers, ReLU, max-pooling, and fully connected layers.
Optimized using SGD with momentum and weight decay.
#### Hyperparameter Tuning:

Tested learning rates (0.05, 0.08, 0.11), momentum (0.7, 0.8), and weight decay (0.001).
Best configuration: LR=0.11, Momentum=0.7, Weight Decay=0.001.
#### Performance:

Validation Accuracy: ~50%.
Test Metrics:
Accuracy: 49.26%, Precision: 48.71%, Recall: 41.50%, F1 Score: 40.56%.

### Strengths:
ClearML enabled effective experiment tracking.
Learning rate decay improved convergence.
Achieved moderate performance with a simple CNN.

### Challenges:
Limited accuracy (~49%) indicates potential class imbalance or insufficient model complexity.
Variability in precision and recall suggests underrepresented classes.